<a href="https://colab.research.google.com/github/teamsimcoemmai/mmai894/blob/main/Final%20Project/DenseNet_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import numpy as np

import cv2 as cv

import os
import random

import tensorflow as tf

from shutil import copyfile

from tqdm import tqdm

from fastai.vision import *
from fastai.metrics import error_rate
from fastai import *
import cv2 as cv
import numpy as np
import pandas as pd
import scipy.io as sio

from sklearn.metrics import precision_recall_fscore_support

import shutil

In [ ]:
def random_seed(seed_value, use_cuda):  #gleaned from multiple forum posts
    np.random.seed(seed_value) # cpu vars
    torch.manual_seed(seed_value) # cpu  vars
    random.seed(seed_value) # Python
    if use_cuda: torch.cuda.manual_seed_all(seed_value) # gpu 


random_seed(42,True)

data = ImageDataBunch.from_folder('/content/gdrive/MyDrive/Simcoe/894/Car dataset/new','train_set','val_set','test_set',
                                  seed=42,
                                  ds_tfms=get_transforms(do_flip=True, 
                                                         flip_vert=True, 
                                                         p_affine=0.75, # Default value
                                                         max_rotate=5.0, 
                                                         max_zoom=1.1,
                                                         p_lighting=0.75, # Defalut value 
                                                         max_lighting=0.2
                                                         ),
                                  size=224,
                                  bs=32).normalize(imagenet_stats)

In [ ]:
df_test = pd.read_csv('/content/gdrive/MyDrive/Simcoe/894/Car dataset/df_test_new.csv')

In [ ]:

def test_predict(df_test):

  '''
  Args
  df_test: Test data df

  Return:
  Data frame with predicted and actual for each image from the test data set   
  '''

#Predict from test data set
  prediction,y = learn.get_preds(DatasetType.Test)

#Map predictions to image name on test data set
  preds = []
  dataIndex = []
  num_preds = len(prediction)
  for i in range(num_preds):
    p = learn.predict(data.test_ds.x[i])
    preds.append(str(p[0]))
    dataIndex.append(str(learn.data.test_ds.items[i]).split('/')[-1])

#Convert predictions from string to Int 
  preds = list(map(int, preds))

#Zip image name and predictions
  pred_tup = list(zip(dataIndex,preds))

#Create data frame from image name and predictions
  df_pred = pd.DataFrame(pred_tup,columns=['img_name','pred_class'])

#Load test data set - Data has already been pre processed and uploaded into drive
  actual_tup = df_test[['img_name','class']].to_records(index=False)

# Join preictions test daframe and predictions data fram
  df_valid = df_test.merge(df_pred, left_on='img_name', right_on='img_name')

  feature_list = ['img_name','class','pred_class']
  df_valid = df_valid[feature_list]

  return df_valid




In [ ]:
def precision_recall_fscore(df):

  '''
  Function to calculate precision, recall, fscore
  '''

  precision, recall, fscore, support = precision_recall_fscore_support(df['class'].values,
                                                                     df['pred_class'].values, 
                                                                     average = 'weighted')
  
  return precision, recall, fscore


In [ ]:
def calc_accuracy(df):

  '''
  Function to calculate accuracy
  '''

  # Find accuracy from test data set
  df['diff'] = np.where(df['class'] == df['pred_class'],1,0)
  accuracy = df['diff'].sum()/df.shape[0]*100

  return accuracy

In [ ]:
learn = create_cnn(data, 
                   models.densenet121,
                   metrics=[accuracy])

/usr/local/lib/python3.7/dist-packages/fastai/vision/learner.py:109: UserWarning: `create_cnn` is deprecated and is now named `cnn_learner`.
  warn("`create_cnn` is deprecated and is now named `cnn_learner`.")
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth


In [ ]:
from fastai.callbacks import EarlyStoppingCallback

# Callback to avoid over fitting
callback = EarlyStoppingCallback(learn,
                                  monitor = 'accuracy',
                                  patience = 2)                                               
                                

In [ ]:
from fastai.callbacks import *

# Transfer learning from pretrained models
#Hyper paramer tuning - fit_one_cycle method varies learing rate to get the best accuracy
learn.fit_one_cycle(10,
                    callbacks = callback
                    )

epoch,train_loss,valid_loss,accuracy,time
0,5.546669,4.323173,0.117381,45:13
1,3.796674,2.826779,0.332784,02:53
2,2.689441,1.932150,0.503295,02:48
3,2.079059,1.540993,0.600494,02:48
4,1.632178,1.277961,0.655272,02:47
5,1.315103,1.079730,0.708402,02:49
6,1.094302,0.939369,0.745881,02:52
7,0.822349,0.861579,0.772241,02:51
8,0.741728,0.820492,0.773888,02:49
9,0.693596,0.813074,0.776771,02:47


In [ ]:
learn.save('/content/gdrive/MyDrive/Simcoe/894/saved_models/densenet121')

In [ ]:
learn.load('/content/gdrive/MyDrive/Simcoe/894/saved_models/densenet121')

Learner(data=ImageDataBunch;

Train: LabelList (11330 items)
x: ImageList
Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224)
y: CategoryList
152,152,152,152,152
Path: /content/gdrive/MyDrive/Simcoe/894/Car dataset/new;

Valid: LabelList (2428 items)
x: ImageList
Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224)
y: CategoryList
192,192,192,192,192
Path: /content/gdrive/MyDrive/Simcoe/894/Car dataset/new;

Test: LabelList (2427 items)
x: ImageList
Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224)
y: EmptyLabelList
,,,,
Path: /content/gdrive/MyDrive/Simcoe/894/Car dataset/new, model=Sequential(
  (0): Sequential(
    (0): Sequential(
      (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu0): ReLU(inpla

In [ ]:
df_test_predict = test_predict(df_test)

   Score after Transfer learing
   

In [ ]:
precision, recall, fscore = precision_recall_fscore(df_test_predict)

print("Precision : " + str(precision))
print("Recall: " + str(recall))
print("F1 Score : " + str(fscore))

Precision : 0.7948744333518215
Recall: 0.7754429336629584
F1 Score : 0.7750791360015669


In [ ]:
accuracy = calc_accuracy(df_test_predict)

print("Accuracy :" + str(accuracy))

Accuracy :77.54429336629583


In [ ]:
#Unfreeze model

learn.unfreeze()

#Hyper paramer tuning - fit_one_cycle method varies learing rate to get the best accuracy
learn.fit_one_cycle(10, max_lr = slice(2e-5, 2e-4), wd = 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.710178,0.804424,0.777595,02:58
1,0.619971,0.766217,0.793657,02:57
2,0.625443,0.722068,0.797364,02:57
3,0.538585,0.648198,0.819193,02:57
4,0.486170,0.624008,0.829077,02:57
5,0.384024,0.581966,0.839374,03:00
6,0.372474,0.548932,0.846376,02:57
7,0.308978,0.525277,0.848435,02:57
8,0.294959,0.523383,0.851730,02:57
9,0.296644,0.521095,0.852142,02:57


In [ ]:
learn.save('/content/gdrive/MyDrive/Simcoe/894/saved_models/densenet121_unfreeze')

In [ ]:
learn.load('/content/gdrive/MyDrive/Simcoe/894/saved_models/densenet121_unfreeze')

Learner(data=ImageDataBunch;

Train: LabelList (11330 items)
x: ImageList
Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224)
y: CategoryList
152,152,152,152,152
Path: /content/gdrive/MyDrive/Simcoe/894/Car dataset/new;

Valid: LabelList (2428 items)
x: ImageList
Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224)
y: CategoryList
192,192,192,192,192
Path: /content/gdrive/MyDrive/Simcoe/894/Car dataset/new;

Test: LabelList (2427 items)
x: ImageList
Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224)
y: EmptyLabelList
,,,,
Path: /content/gdrive/MyDrive/Simcoe/894/Car dataset/new, model=Sequential(
  (0): Sequential(
    (0): Sequential(
      (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu0): ReLU(inpla

In [ ]:
df_test_predict = test_predict(df_test)

Scores after unfreezing

In [ ]:
precision, recall, fscore = precision_recall_fscore(df_test_predict)

print("Precision : " + str(precision))
print("Recall: " + str(recall))
print("F1 Score : " + str(fscore))

Precision : 0.9020075620330947
Recall: 0.8953440461475072
F1 Score : 0.894663473401842


In [ ]:
accuracy = calc_accuracy(df_test_predict)

print("Accuracy :" + str(accuracy))

Accuracy :89.53440461475071
